# Complete Cars Price Prediction - Interactive Regression Model

**Goal:** Predict the actual price of cars in USD

---

# 1. Import All Libraries

In [2]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import pickle
import warnings
import kagglehub
import os
warnings.filterwarnings('ignore')

# 2. Load Raw Data

In [3]:
path = kagglehub.dataset_download("abdulmalik1518/cars-datasets-2025")

csv_path = os.path.join(path, "Cars Datasets 2025.csv")

df = pd.read_csv(csv_path, encoding="latin1")

print(f"Total rows: {len(df):,}")
print(f"Total columns: {len(df.columns)}")
print(f"\nFirst 5 rows:")
df.head()

Total rows: 1,218
Total columns: 11

First 5 rows:


,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


In [4]:
print("Data Quality Check:")
print(f"\nMissing values per column:")
print(df.isnull().sum())
print(f"\nDuplicate rows: {df.duplicated().sum()}")

Data Quality Check:

Missing values per column:
Company Names                0
Cars Names                   0
Engines                      0
CC/Battery Capacity          3
HorsePower                   0
Total Speed                  0
Performance(0 - 100 )KM/H    6
Cars Prices                  0
Fuel Types                   0
Seats                        0
Torque                       1
dtype: int64

Duplicate rows: 4


# 3. Data Cleaning

## 3.1 Remove Duplicates

In [6]:
df = df.drop_duplicates()

print(f"Duplicates removed")
print(f"Rows now: {len(df):,}")

Duplicates removed
Rows now: 1,214


## 3.2 Fix Company Names

In [7]:
df['Company Names'] = df['Company Names'].str.title()

print("Company names standardized")
print(f"Unique companies: {df['Company Names'].nunique()}")

Company names standardized
Unique companies: 33


## 3.3 Clean All Numeric Features

In [9]:
def clean_horsepower(text):
    if pd.isna(text):
        return None
    text = str(text).lower().replace('hp', '').replace(',', '').strip()
    if '-' in text:
        parts = text.split('-')
        low = float(parts[0].strip())
        high = float(parts[1].strip())
        return (low + high) / 2
    try:
        return float(text)
    except:
        return None

def clean_speed(text):
    if pd.isna(text):
        return None
    text = str(text).lower().replace('km/h', '').replace('km', '').strip()
    try:
        return float(text)
    except:
        return None

def clean_acceleration(text):
    if pd.isna(text):
        return None
    text = str(text).lower().replace('sec', '').strip()
    try:
        return float(text)
    except:
        return None

def clean_price(text):
    if pd.isna(text):
        return None
    text = str(text).replace('$', '').replace(',', '').replace(' ', '')
    if '-' in text:
        parts = text.split('-')
        try:
            low = float(parts[0].strip())
            high = float(parts[1].strip())
            return (low + high) / 2
        except:
            return None
    try:
        return float(text)
    except:
        return None

def clean_torque(text):
    if pd.isna(text):
        return None
    text = str(text).lower().replace('nm', '').replace(',', '').replace(' ', '')
    if '-' in text:
        parts = text.split('-')
        try:
            low = float(parts[0].strip())
            high = float(parts[1].strip())
            return (low + high) / 2
        except:
            return None
    try:
        return float(text)
    except:
        return None

def clean_seats(text):
    if pd.isna(text):
        return None
    text = str(text).strip()
    if '+' in text:
        parts = text.split('+')
        try:
            return sum(int(p.strip()) for p in parts)
        except:
            return None
    if '-' in text or '–' in text:
        parts = text.replace('–', '-').split('-')
        try:
            return int(parts[-1].strip())
        except:
            return None
    try:
        return int(text)
    except:
        return None

def simplify_fuel(text):
    if pd.isna(text):
        return 'Unknown'
    text = str(text).lower()
    if 'electric' in text or 'ev' in text:
        return 'Electric'
    elif 'hybrid' in text or 'plug' in text:
        return 'Hybrid'
    elif 'diesel' in text:
        return 'Diesel'
    elif 'petrol' in text or 'gas' in text:
        return 'Petrol'
    else:
        return 'Other'

df['Horsepower'] = df['HorsePower'].apply(clean_horsepower)
df['Speed'] = df['Total Speed'].apply(clean_speed)
df['Acceleration'] = df['Performance(0 - 100 )KM/H'].apply(clean_acceleration)
df['Price'] = df['Cars Prices'].apply(clean_price)
df['Torque'] = df['Torque'].apply(clean_torque)
df['Seats_Clean'] = df['Seats'].apply(clean_seats)
df['Fuel_Type'] = df['Fuel Types'].apply(simplify_fuel)

print("All features cleaned")

All features cleaned


## 3.4 Fill Missing Values

In [10]:
df['Horsepower'] = df['Horsepower'].fillna(df['Horsepower'].median())
df['Speed'] = df['Speed'].fillna(df['Speed'].median())
df['Acceleration'] = df['Acceleration'].fillna(df['Acceleration'].median())
df['Price'] = df['Price'].fillna(df['Price'].median())
df['Torque'] = df['Torque'].fillna(df['Torque'].median())
df['Seats_Clean'] = df['Seats_Clean'].fillna(df['Seats_Clean'].median())

print("Missing values filled")

Missing values filled


## 3.5 Create Clean Dataset

In [11]:
df_clean = df[[
    'Company Names',
    'Cars Names',
    'Fuel_Type',
    'Horsepower',
    'Torque',
    'Speed',
    'Acceleration',
    'Seats_Clean',
    'Price'
]].copy()

df_clean.columns = [
    'Company',
    'Model',
    'Fuel_Type',
    'Horsepower',
    'Torque_Nm',
    'Max_Speed_kmh',
    'Acceleration_0_100_sec',
    'Seats',
    'Price_USD'
]

print("Clean dataset created")

Clean dataset created


In [12]:
print("\nClean Data Sample:")
df_clean.head(10)


Clean Data Sample:


,Company,Model,Fuel_Type,Horsepower,Torque_Nm,Max_Speed_kmh,Acceleration_0_100_sec,Seats,Price_USD
0,Ferrari,SF90 STRADALE,Hybrid,963.0,800.0,340.0,2.5,2.0,1100000.0
1,Rolls Royce,PHANTOM,Petrol,563.0,900.0,250.0,5.3,5.0,460000.0
2,Ford,KA+,Petrol,77.5,120.0,165.0,10.5,5.0,13500.0
3,Mercedes,GT 63 S,Petrol,630.0,900.0,250.0,3.2,4.0,161000.0
4,Audi,AUDI R8 Gt,Petrol,602.0,560.0,320.0,3.6,2.0,253290.0
5,Bmw,Mclaren 720s,Petrol,710.0,770.0,341.0,2.9,2.0,499000.0
6,Aston Martin,VANTAGE F1,Petrol,656.0,685.0,314.0,3.6,2.0,193440.0
7,Bentley,Continental GT Azure,Petrol,550.0,900.0,318.0,4.0,4.0,311000.0
8,Lamborghini,VENENO ROADSTER,Petrol,750.0,690.0,356.0,2.9,2.0,4500000.0
9,Ferrari,F8 TRIBUTO,Petrol,710.0,770.0,340.0,2.9,2.0,280000.0


# 4. Exploratory Data Analysis (EDA)

## 4.1 Statistical Summary

In [13]:
print("Statistical Summary:")
df_clean.describe()

Statistical Summary:


,Horsepower,Torque_Nm,Max_Speed_kmh,Acceleration_0_100_sec,Seats,Price_USD
count,1214.000000,1214.000000,1214.000000,1214.000000,1214.000000,1.214000e+03
mean,303.642092,507.450577,216.499176,7.565321,4.853377,1.383136e+05
std,209.554466,1050.300898,53.003441,3.301451,1.497979,7.118983e+05
min,26.000000,45.000000,80.000000,1.900000,1.000000,4.000000e+03
25%,155.000000,250.000000,180.000000,5.300000,4.000000,2.800000e+04
50%,255.000000,371.500000,200.000000,7.100000,5.000000,4.280750e+04
75%,400.000000,560.000000,250.000000,9.500000,5.000000,7.000000e+04
max,1850.000000,15590.000000,500.000000,35.000000,20.000000,1.800000e+07


## 4.2 Price Distribution (Target Variable)

In [14]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Price Distribution', 'Log Price Distribution')
)

fig.add_trace(
    go.Histogram(x=df_clean['Price_USD'], nbinsx=50, name='Price', marker_color='green'),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=np.log1p(df_clean['Price_USD']), nbinsx=50, name='Log Price', marker_color='blue'),
    row=1, col=2
)

median_price = df_clean['Price_USD'].median()
fig.add_vline(x=median_price, line_dash="dash", line_color="red", row=1, col=1,
              annotation_text=f"Median: ${median_price:,.0f}")

fig.update_xaxes(title_text="Price (USD)", row=1, col=1)
fig.update_xaxes(title_text="Log(Price)", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=2)

fig.update_layout(height=500, showlegend=False, title_text="Price Distribution Analysis")
fig.show()

print(f"\nPrice Statistics:")
print(f"Min: ${df_clean['Price_USD'].min():,.0f}")
print(f"Max: ${df_clean['Price_USD'].max():,.0f}")
print(f"Mean: ${df_clean['Price_USD'].mean():,.0f}")
print(f"Median: ${df_clean['Price_USD'].median():,.0f}")


Price Statistics:
Min: $4,000
Max: $18,000,000
Mean: $138,314
Median: $42,808


## 4.3 Fuel Type Distribution

In [15]:
fuel_counts = df_clean['Fuel_Type'].value_counts().reset_index()
fuel_counts.columns = ['Fuel_Type', 'Count']

fig = px.bar(fuel_counts, x='Fuel_Type', y='Count',
             title='Fuel Type Distribution',
             color='Fuel_Type',
             text='Count',
             color_discrete_sequence=px.colors.qualitative.Set3)

fig.update_traces(textposition='outside')
fig.update_layout(showlegend=False, height=500)
fig.update_xaxes(title='Fuel Type')
fig.update_yaxes(title='Count')
fig.show()

## 4.4 Top 15 Companies

In [16]:
top_companies = df_clean['Company'].value_counts().head(15).reset_index()
top_companies.columns = ['Company', 'Count']

fig = px.bar(top_companies, x='Count', y='Company', orientation='h',
             title='Top 15 Car Companies',
             text='Count',
             color='Count',
             color_continuous_scale='Blues')

fig.update_traces(textposition='outside')
fig.update_layout(height=600, showlegend=False, yaxis={'categoryorder':'total ascending'})
fig.update_xaxes(title='Number of Cars')
fig.update_yaxes(title='Company')
fig.show()

## 4.5 Correlation Analysis

In [18]:
numeric_cols = ['Horsepower', 'Torque_Nm', 'Max_Speed_kmh', 'Acceleration_0_100_sec', 'Price_USD']
correlation = df_clean[numeric_cols].corr()

fig = go.Figure(data=go.Heatmap(
    z=correlation.values,
    x=correlation.columns,
    y=correlation.columns,
    colorscale='RdBu',
    zmid=0,
    text=correlation.values.round(2),
    texttemplate='%{text}',
    textfont={"size": 12},
    colorbar=dict(title="Correlation")
))

fig.update_layout(
    title='Correlation Matrix (Including Price)',
    height=600,
    width=700
)
fig.show()

The results show that price has a moderate positive correlation with horsepower (0.54) and max speed (0.39), meaning cars with more power and higher top speeds tend to be more expensive. Torque shows a very weak relationship with price (0.10), while acceleration time has a slight negative correlation (-0.20), indicating that faster-accelerating cars (lower seconds) are generally priced higher.

## 4.6 Feature vs Price Relationships

In [19]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Horsepower vs Price', 'Max Speed vs Price', 'Torque vs Price', 'Acceleration vs Price')
)

fig.add_trace(
    go.Scatter(x=df_clean['Horsepower'], y=df_clean['Price_USD'], mode='markers',
               marker=dict(color='blue', size=5, opacity=0.5), name='Horsepower'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df_clean['Max_Speed_kmh'], y=df_clean['Price_USD'], mode='markers',
               marker=dict(color='green', size=5, opacity=0.5), name='Speed'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=df_clean['Torque_Nm'], y=df_clean['Price_USD'], mode='markers',
               marker=dict(color='red', size=5, opacity=0.5), name='Torque'),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=df_clean['Acceleration_0_100_sec'], y=df_clean['Price_USD'], mode='markers',
               marker=dict(color='orange', size=5, opacity=0.5), name='Acceleration'),
    row=2, col=2
)

fig.update_xaxes(title_text="Horsepower", row=1, col=1)
fig.update_xaxes(title_text="Max Speed (km/h)", row=1, col=2)
fig.update_xaxes(title_text="Torque (Nm)", row=2, col=1)
fig.update_xaxes(title_text="Acceleration 0-100 (sec)", row=2, col=2)

fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
fig.update_yaxes(title_text="Price (USD)", row=1, col=2)
fig.update_yaxes(title_text="Price (USD)", row=2, col=1)
fig.update_yaxes(title_text="Price (USD)", row=2, col=2)

fig.update_layout(height=800, showlegend=False, title_text="Features vs Price Relationships")
fig.show()

## 4.7 Price by Fuel Type

In [20]:
fig = px.box(df_clean, x='Fuel_Type', y='Price_USD',
             title='Price Distribution by Fuel Type',
             color='Fuel_Type',
             color_discrete_sequence=px.colors.qualitative.Set2)

fig.update_layout(showlegend=False, height=600)
fig.update_xaxes(title='Fuel Type')
fig.update_yaxes(title='Price (USD)')
fig.show()

print("\n💰 Average Price by Fuel Type:")
avg_price = df_clean.groupby('Fuel_Type')['Price_USD'].mean().sort_values(ascending=False)
for fuel, price in avg_price.items():
    print(f"   {fuel}: ${price:,.0f}")


💰 Average Price by Fuel Type:
   Petrol: $161,551
   Hybrid: $133,892
   Electric: $67,599
   Other: $46,633
   Diesel: $41,440


The results show clear price differences across fuel types. Petrol cars have the highest average price at about $161,551, followed by hybrids at around $133,892. Electric cars are priced noticeably lower on average at about $67,599, while “Other” fuel types average $46,633. Diesel cars have the lowest average price at roughly $41,440, indicating they tend to be the most budget-friendly among the categories.

### *Summary*

The dataset contains 1,214 cars, with prices ranging from $4,000 to as high as $18,000,000. The mean price is $138,314, while the median is much lower at $42,808, suggesting the presence of very expensive outliers. Petrol is the most common fuel type in the dataset. When looking at price relationships, horsepower shows the strongest positive correlation with price (0.541), followed by max speed (0.388), while torque has only a very weak correlation (0.096).

# 5. Prepare Data for Regression

## 5.1 Drop Unnecessary Columns

In [23]:
df_model = df_clean.drop(columns=['Model'])

print("Dropped column: 'Model'")

Dropped column: 'Model'


## 5.2 Separate Features (X) and Target (y)

In [24]:
X = df_model.drop(columns=['Price_USD'])
y = df_model['Price_USD']

print("Features and target separated")
print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")

Features and target separated
Features (X): (1214, 7)
Target (y): (1214,)


## 5.3 Identify Feature Types

In [25]:
categorical_features = ['Company', 'Fuel_Type']
numerical_features = ['Horsepower', 'Torque_Nm', 'Max_Speed_kmh', 'Acceleration_0_100_sec', 'Seats']

print("Feature types identified")

Feature types identified


# 6. Column Transformer

In [26]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

print("Column transformer created")

Column transformer created


# 7. Train / Test Split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Data split complete")
print(f"Training: {X_train.shape[0]} samples")
print(f"Test: {X_test.shape[0]} samples")

Data split complete
Training: 971 samples
Test: 243 samples



# 8. Model Building with GridSearchCV

In [29]:
pipe = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', RandomForestRegressor(random_state=123))
])

param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [10, 20, 30, None],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__max_features': ['sqrt', 'log2']
}

cv_strategy = KFold(n_splits=5, shuffle=True, random_state=123)

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=cv_strategy,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

print("\nTraining complete")
print(f"\nBest Parameters: {grid.best_params_}")
print(f"Best CV MAE: ${-grid.best_score_:,.0f}")

best_model = grid.best_estimator_

Fitting 5 folds for each of 216 candidates, totalling 1080 fits

Training complete

Best Parameters: {'model__max_depth': 20, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 1, 'model__min_samples_split': 10, 'model__n_estimators': 200}
Best CV MAE: $73,986


# 9. Model Evaluation with Interactive Plots

## 9.1 Make Predictions

In [30]:
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("Predictions made")
print(f"\nPerformance Metrics:")
print(f"MAE: ${mae:,.0f}")
print(f"RMSE: ${rmse:,.0f}")
print(f"R² Score: {r2:.4f}")

Predictions made

Performance Metrics:
MAE: $42,378
RMSE: $211,681
R² Score: 0.6930


## 9.2 Actual vs Predicted

In [33]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y_test,
    y=y_pred,
    mode='markers',
    marker=dict(size=8, color='blue', opacity=0.6),
    name='Predictions'
))

min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    line=dict(color='red', dash='dash', width=2),
    name='Perfect Prediction'
))

fig.update_layout(
    title=f'Actual vs Predicted Prices (R² = {r2:.4f})',
    xaxis_title='Actual Price (USD)',
    yaxis_title='Predicted Price (USD)',
    height=600
)
fig.show()

The red line shows perfect predictions. Points close to the line mean the model predicted well, while points far from the line mean the predictions were inaccurate.

## 9.3 Sample Predictions

In [36]:
errors = y_test - y_pred

results_df = pd.DataFrame({
    'Actual_Price': y_test.values,
    'Predicted_Price': y_pred,
    'Error': errors.values,
})

print("Sample Predictions:")
print(results_df.head(10).to_string())

Sample Predictions:
   Actual_Price  Predicted_Price          Error
0       33900.0     42682.262864   -8782.262864
1       82000.0     80360.196315    1639.803685
2      518000.0    976879.068472 -458879.068472
3       34000.0     35548.350008   -1548.350008
4       18000.0     20594.497493   -2594.497493
5       22500.0     25460.370345   -2960.370345
6       22000.0     28090.337686   -6090.337686
7       79000.0     80980.056952   -1980.056952
8      114000.0    137999.271755  -23999.271755
9      355000.0    405054.202892  -50054.202892


# Final Summary

This regression project used a dataset of 1,214 cars and seven features to predict car prices using a RandomForestRegressor model. The model achieved a Mean Absolute Error of about $42,378, meaning predictions are usually off by that amount on average, and an RMSE of $211,681, showing that some predictions have much larger errors due to expensive outliers. The R² score of 0.69 indicates the model explains around 69% of the variation in car prices.